# Śledzenie rozgyrwki bilarda

## Import niezbędnych bibliotek

In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
from collections import deque
# import PIL
# import os
# import sys
# import glob
# import random

# from pprint import pprint
from ipywidgets import Video

import PIL
from IPython.display import display
# from PIL.ExifTags import TAGS

def imshow(a):
  a = a.clip(0, 255).astype('uint8')
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(PIL.Image.fromarray(a))

## Analiza wideo

### Wczytanie wideo i jego podstawowych właściwości

In [15]:
video = cv2.VideoCapture('data/9-ball/video6.mov')
if video.isOpened():
    print('Movie loaded :)')

width = int(video.get(3))
height = int(video.get(4))
print(f'Wysokość i szerokość obrazu: {height} x {width}')

fps = video.get(cv2.CAP_PROP_FPS)
print(f'Liczba klatek na sekundę: {fps}')

Movie loaded :)
Wysokość i szerokość obrazu: 1054 x 1924
Liczba klatek na sekundę: 59.891696750902526


### Wyświetlenie analizowanego wideo

In [16]:
Video.from_file('data/9-ball/video1.mov')

Video(value=b'\x00\x00\x00\x14ftypqt  \x00\x00\x00\x00qt  \x00\x00\x00\x08wide\x01\x02\xee\xc3mdat\x00\x01\x03…

### Poszukiwanie współrzędnych bil na filmie

### Wykorzystanie algorytmu CSRT do śledzenia znalezionych bili na filmie

In [17]:
def create_tracker(tracker_type):
  tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

  if tracker_type == 'BOOSTING':
      return cv2.legacy.TrackerBoosting_create()
  if tracker_type == 'MIL':
      return cv2.legacy.TrackerMIL_create()
  if tracker_type == 'KCF':
      return cv2.legacy.TrackerKCF_create()
  if tracker_type == 'TLD':
      return cv2.legacy.TrackerTLD_create()
  if tracker_type == 'MEDIANFLOW':
      return cv2.legacy.TrackerMedianFlow_create()
  if tracker_type == 'GOTURN':
      return cv2.TrackerGOTURN_create()
  if tracker_type == 'MOSSE':
      return cv2.legacy.TrackerMOSSE_create()
  if tracker_type == "CSRT":
      return cv2.TrackerCSRT_create()


In [18]:
def draw_bbox(frame, bbox, color=(255, 255, 255)):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)
    
def circle_to_rectangle(circle, radius):
    return (int(circle[0]-radius), int(circle[1]-radius), int(2*radius),  int(2*radius))

def collision(c1, c2):
    if (abs(c1[0] - c2[0])) < 80 and (abs(c1[1] - c2[1])) < 80:
        return True
    else:
        return False

def count_centroid(b):
    return [ int(b[0] + b[2]/2), int(b[1] + b[3]/2) ]

def count_distance(c1, c2):
    clash = collision(c1, c2)
    if clash:
        x = (c1[0] + c2[0])/2
        y = (c1[1] + c2[1])/2
        return [int(x), int(y)]
    else:
        return 0

In [19]:
video = cv2.VideoCapture('movies/video4.mov')
if video.isOpened():
    print('Movie loaded :)')

width = int(video.get(3))
height = int(video.get(4))

fps = video.get(cv2.CAP_PROP_FPS)
print(f'Liczba klatek na sekundę: {fps}')


Movie loaded :)
Liczba klatek na sekundę: 57.634408602150536


In [20]:
number_of_frames = 5

In [54]:
video.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, frame = video.read()

def distance(a, b):
  return np.array((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

filter = np.array([
  [-1, -1, -1], 
  [-1, 9, -1], 
  [-1, -1, -1]
])

HISTORY = 10
MOV_TOLERANCE = 3

sharpen_img_1=cv2.filter2D(frame,-1,filter)
cimg = cv2.cvtColor(sharpen_img_1,cv2.COLOR_BGR2GRAY)
circles = cv2.HoughCircles(cimg,cv2.HOUGH_GRADIENT,1,20,
                            param1=60,param2=20,minRadius=13,maxRadius=22)
circles = np.uint16(np.around(circles))
max_radius = np.max(circles[0,:,2]) + 10

blur = cv2.medianBlur(frame,7)
cimg_blur = cv2.cvtColor(blur,cv2.COLOR_BGR2GRAY)
circles_blur = cv2.HoughCircles(cimg,cv2.HOUGH_GRADIENT,1,30,
                        param1=60,param2=20,minRadius=35,maxRadius=50)
pockets = np.uint16(np.around(circles))


bills = []

for c in circles[0]:
  bills.append(create_tracker('CSRT'))
  bills[-1].init(frame, circle_to_rectangle(c, max_radius))

frames_track = np.zeros((len(bills), 2))
frames_number = 0
last_frames_pos = np.zeros((HISTORY,len(bills),  2), dtype=np.int)
last_frames_v = np.zeros((HISTORY, len(bills),  2), dtype=np.int)

def calculate_velocity(x, y, fps, n_frames):
  return np.sqrt(x**2 + y**2)/(n_frames/fps)

video_tracker = cv2.VideoWriter('video_tracker_4.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
video.set(cv2.CAP_PROP_POS_FRAMES, 0)

while video.isOpened():
    ret, frame = video.read()
    
    if ret:
      pos = np.zeros((len(bills), 2), dtype=np.int)
      vel = np.zeros((len(bills),2 ), dtype=np.int)

      for idx, b in enumerate(bills):
        ok, bbox = b.update(frame)
        pos[idx] = count_centroid(bbox)

        if distance(pos[idx], last_frames_pos[-1][idx]) > MOV_TOLERANCE:

          vel[idx] = np.array([
            pos[idx][0] - last_frames_pos[0][idx][0], pos[idx][1] - last_frames_pos[0][idx][1]
          ])
        else:
          vel[idx] = np.array([0,0])

        last_frames_pos = np.concatenate([last_frames_pos, [pos]], axis=0)
        last_frames_v = np.concatenate([last_frames_v, [vel]], axis=0)

        last_frames_pos = np.delete(last_frames_pos, 0, axis=0)
        last_frames_v = np.delete(last_frames_v, 0, axis=0)

        # cv2.putText(frame, 'ID', (res[0], res[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        if ok: draw_bbox(frame, bbox, (0, 255, 0))

        bill_pos = last_frames_pos[-1][0]
        bill_v = last_frames_v[-1][0]

        velocity = calculate_velocity(bill_v[0], bill_v[1], fps, HISTORY)
        # vectors[i] = [box[-1][0] - box[0][0], box[-1][1] - box[0][1]]
        # velocity = count_velocity(vectors[i][0], vectors[i][1], fps, number_of_frames)
        # print(vectors[i])
        cv2.putText(frame, f'{velocity:.4f}', (200, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

          # for j, bo in enumerate(last_frames[2]):
          #   if (i != j):
          #     result = count_distance(box, bo)
          #     if result != 0:
          #       if ~np.array_equal(last_frames[0][i],last_frames[-1][i]):
          #         if frames_track[i][0] != j:
          #           frames_track[i] = [j, frames_number]
          #         if (frames_track[i][0] == j and frames_number - frames_track[i][1] < 180):
          #           # print(i,j)
          #           cv2.putText(frame, 'Collision', (result[0], result[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)


      video_tracker.write(frame)
    else:
        break
    
  
    frames_number += 1

    # if frames_number > 200:
      # break

video_tracker.release()

/home/czewian/.local/lib/python3.7/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/czewian/.local/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional i

In [55]:
!ffmpeg -hide_banner -loglevel error -i video_tracker_4.avi -y video_tracker_4.mp4

In [56]:
Video.from_file('video_tracker_4.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x0f\xb3&mdat\x00\x…

In [23]:
# !ffmpeg -hide_banner -loglevel error -i video_tracker_4.avi -y video_tracker_4.mp4